In [7]:
from collections import Counter
from itertools import product

class DPCFeatureExtraction:
    def __init__(self):
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    def calculate_dpc_features(self, sequence):
        """
        Calculate the DPC features for a given protein sequence.

        :param sequence: The protein sequence as a string.
        :return: A list of DPC feature values.
        """
        # Generate all possible dipeptide combinations
        dipeptides = [''.join(dipeptide) for dipeptide in product(self.amino_acids, repeat=2)]

        # Initialize a dictionary to store the counts of each dipeptide
        dipeptide_counts = {dipeptide: 0 for dipeptide in dipeptides}

        # Count the occurrences of each dipeptide in the sequence
        for i in range(len(sequence) - 1):
            dipeptide = sequence[i:i+2]
            if dipeptide in dipeptide_counts:
                dipeptide_counts[dipeptide] += 1

        # Calculate the sequence length
        sequence_length = len(sequence)

        # Calculate the DPC feature values using the formula
        dpc_features = [dipeptide_counts[dipeptide] / (sequence_length - 1) for dipeptide in dipeptides]

        return dpc_features

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, matthews_corrcoef
from sklearn.model_selection import LeaveOneOut
import numpy as np

# Load the dataset
data = pd.read_excel('../data/Final_2Sm_modified_with_sequences.xlsx')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder to the folding_type column and transform it to numeric labels
data['folding_type'] = label_encoder.fit_transform(data['folding_type'])

# Now, when you extract labels for model training:
labels = data['folding_type'].values

# Initialize the FeatureExtraction class
feature_extraction = DPCFeatureExtraction()

# Feature extraction using AAC with length
features = np.array([feature_extraction.calculate_dpc_features(seq) for seq in data['sequence']])

# SVM with Leave-One-Out Cross-Validation (LOOCV)
loo = LeaveOneOut()
y_true, y_pred = [], []
for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred.append(clf.predict(X_test)[0])
    y_true.append(y_test[0])

In [6]:
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'DPC')
cm.evaluate()

Confusion Matrix: $DPC
[[89  0]
 [52  0]]

Accuracy (ACC): 0.63
Matthews Correlation Coefficient (MCC): 0.00

Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        89
           1       0.00      0.00      0.00        52

    accuracy                           0.63       141
   macro avg       0.32      0.50      0.39       141
weighted avg       0.40      0.63      0.49       141


# Random Forest implementation with hyperparameter tuning

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Grid search with cross validation setup
grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=3, scoring='accuracy')

# Fit the grid search to find the best parameters
grid_search.fit(features, labels)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters: ", best_params)
print("Best score: ", best_score)

# Using the best parameters with LOOCV
best_rf = RandomForestClassifier(**best_params, random_state=42)
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    best_rf.fit(X_train, y_train)
    y_pred.append(best_rf.predict(X_test)[0])
    y_true.append(y_test[0])

Best parameters:  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best score:  0.7661904761904761


In [12]:
# Evaluate the model
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'KAAC')
cm.evaluate()

Confusion Matrix: $KAAC
[[84  5]
 [34 18]]

Accuracy (ACC): 0.72
Matthews Correlation Coefficient (MCC): 0.38

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.94      0.81        89
           1       0.78      0.35      0.48        52

    accuracy                           0.72       141
   macro avg       0.75      0.64      0.65       141
weighted avg       0.74      0.72      0.69       141
